# Playground for ASFPy module development

In [1]:
import os
import csv
import random
from copy import deepcopy

from asfpy import asfpy

In [2]:
from pathlib import Path

FOLDER = "asfp2020"
EDITORS_FILE = "processed-editors-list-draft.csv"
APPLICANTS_FILE = "processed-applicants-list-draft.csv"
MATCHINGS_FILE = "matchings-list-draft.csv"

path = Path("__file__").parent.absolute()

editors_filename = path.parent.joinpath(FOLDER).joinpath(EDITORS_FILE)
editors = asfpy.read_preprocessed_editors_list_csv(editors_filename)

applicants_filename = path.parent.joinpath(FOLDER).joinpath(APPLICANTS_FILE)
applicants = asfpy.read_preprocessed_applicants_list_csv(applicants_filename)

matchings_filename = path.parent.joinpath(FOLDER).joinpath(MATCHINGS_FILE)

In [3]:
_editors = deepcopy(editors)
_applicants = deepcopy(asfpy.randomize(applicants))

p_applicants = asfpy.prioritize(_applicants, rank_method = asfpy.asfp_rank)

allocation_output = asfpy.allocate(p_applicants, _editors)

# Allocation Output Handling

The output from running the allocation method includes a list of matchings, a list of unmatched applicants by identifier strings, and a list of editors _after_ decrementing capacities.

## Handle saving matchings

In [4]:
matchings = allocation_output["matchings"]
dyads = asfpy.format_matchings(matchings, applicants, editors)
asfpy.write_matchings_list_csv(dyads, matchings_filename)

## New editors by capacity and unmatched applicants

This pattern should be used to re-run the allocation for a given set of editors and applicants. In particular, this would be re-run _if_ editors increase capacity. The applicants remain in the same priority ordering from the first allocation by using `p_applicants`.

In [5]:
new_capacity_editors = allocation_output["editors"]

unmatched_ids = allocation_output["unmatched"]

## Use p_applicants to keep rank order
unmatched_applicants = asfpy.compile_unmatched_applicants(unmatched_ids, p_applicants)
asfpy.allocate(p_applicants, new_capacity_editors)

{'matchings': [{'applicant': 'APP_001', 'editors': ['EDI_024', 'EDI_015']},
  {'applicant': 'APP_002', 'editors': ['EDI_013', 'EDI_015']},
  {'applicant': 'APP_009', 'editors': ['EDI_013', 'EDI_015']},
  {'applicant': 'APP_004', 'editors': ['EDI_030', 'EDI_008']},
  {'applicant': 'APP_010', 'editors': ['EDI_021', 'EDI_006']},
  {'applicant': 'APP_008', 'editors': ['EDI_016', 'EDI_008']},
  {'applicant': 'APP_007', 'editors': ['EDI_030', 'EDI_008']},
  {'applicant': 'APP_003', 'editors': ['EDI_021', 'EDI_009']},
  {'applicant': 'APP_006', 'editors': ['EDI_042', 'EDI_023']},
  {'applicant': 'APP_005', 'editors': ['EDI_012', 'EDI_008']}],
 'unmatched': [],
 'editors': [{'name': 'Jill Holm-Denoma',
   'email': 'jholmden@du.edu',
   'role': 'faculty',
   'capacity': 2,
   'categories': {'Clinical Psychology'},
   'id': 'EDI_001'},
  {'name': 'Melissa Brewster',
   'email': 'Melissa.Brewster@du.edu',
   'role': 'student',
   'capacity': 3,
   'categories': {'Clinical Psychology', 'Developmen